In [1]:
!pip install gradio
!pip install diffusers[torch]
!pip install transformers scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 973.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.

In [2]:
import gradio as gr
import numpy as np
import torch
from PIL import Image
from diffusers import StableDiffusionInpaintPipeline
import base64
from io import BytesIO

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:

# Load the stable diffusion inpainting model
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16,
)
pipe = pipe.to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

...

unet/config.json:   0%|          | 0.00/914 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
def inpaint(image, prompt, x, y, width, height, black_white):
    image_resized = image.resize((512, 512))
    if x == 100 and y == 100 and width == 100 and height == 100:
        mask = np.ones((512, 512), dtype=np.uint8) * 255
    else:
        mask = np.zeros((512, 512), dtype=np.uint8)
        mask[int(y):int(y + height), int(x):int(x + width)] = 255

    mask_image = Image.fromarray(mask)

    try:
        output = pipe(prompt=prompt, image=image_resized, mask_image=mask_image, num_inference_steps=200, guidance_scale=8.5)
        output_image = output.images[0]

        # Convert to black and white if selected
        if black_white:
            output_image = output_image.convert("L")

        return output_image
    except Exception as e:
        print(f"Error during inpainting: {e}")
        return "Error during inpainting"


In [5]:
def update_rectangle(x, y, width, height, image):
    # Convert PIL Image to base64 for HTML embedding
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode()

    return f"""
    <div style='position: relative; width: 512px; height: 512px; border: 2px solid #4A90E2;'>
        <img src='data:image/jpeg;base64,{img_str}' style='width: 100%; height: 100%; object-fit: contain;' />
        <div style='position: absolute; left: {x}px; top: {y}px; width: {width}px; height: {height}px; background-color: rgba(255, 165, 0, 0.5); pointer-events: none;'></div>
    </div>
    """

In [6]:

with gr.Blocks(theme="default") as demo:
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload Image", image_mode="RGB")
        with gr.Column():
            rectangle_output = gr.HTML()
    with gr.Row():
        x_slider = gr.Slider(0, 512, value=100, label="X Position", interactive=True)
        y_slider = gr.Slider(0, 512, value=100, label="Y Position", interactive=True)
        width_slider = gr.Slider(50, 512, value=100, label="Width", interactive=True)
        height_slider = gr.Slider(50, 512, value=100, label="Height", interactive=True)
        prompt_input = gr.Textbox(label="Prompt")
    with gr.Row():
        inpaint_button = gr.Button("Inpaint", elem_id="inpaint-btn")
        black_white_checkbox = gr.Checkbox(label="Black & White", value=False)
    with gr.Row():
        inpainted_image_output = gr.Image(type="pil", label="Output Image")
    inputs = [x_slider, y_slider, width_slider, height_slider, image_input]
    x_slider.change(update_rectangle, inputs, rectangle_output)
    y_slider.change(update_rectangle, inputs, rectangle_output)
    width_slider.change(update_rectangle, inputs, rectangle_output)
    height_slider.change(update_rectangle, inputs, rectangle_output)
    image_input.change(update_rectangle, inputs, rectangle_output)
    inpaint_button.click(inpaint, [image_input, prompt_input, x_slider, y_slider, width_slider, height_slider, black_white_checkbox], inpainted_image_output)

In [7]:
demo.css = """
    #inpaint-btn {
        background-color: #4A90E2;
        color: white;
        border: none;
        cursor: pointer;
    }
    .slider {
        --slider-color: #FF6347;
    }
"""

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ab55dc13a49c4ec1d1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
